In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
# Check if GPU is available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [14]:
# Configuration (for Kaggle environment)
class Config:
    # Model parameters
    SEQUENCE_LENGTH = 30   # T = 30 frames (1–2 seconds at 15–30 fps)
    FEATURE_DIM = 25       # F = feature dimensions
    HIDDEN_DIM = 128
    NUM_LAYERS = 2
    NUM_CLASSES = 6        # Normal, Loitering, Boundary Crossing, Fall, Aggression, Abandoned Object
    DROPOUT = 0.3

    # Training parameters
    BATCH_SIZE = 32
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 100
    EARLY_STOP_PATIENCE = 15
    LR_REDUCE_PATIENCE = 5

    # Data split ratios
    TRAIN_SPLIT = 0.7
    VAL_SPLIT = 0.15
    TEST_SPLIT = 0.15

    # Class names
    CLASS_NAMES = [
        'Normal',
        'Loitering',
        'Boundary_Crossing',
        'Fall',
        'Aggression',
        'Abandoned_Object'
    ]

    # Dataset and results paths (Kaggle auto-mount)
    DATASET_PATH = '/kaggle/input/ucaucf-crime-annotation-dataset'
    MODEL_SAVE_PATH = '/kaggle/working/models'
    RESULTS_PATH= '/kaggle/working/results'


In [15]:
# Feature extraction utilities
def extract_features_from_bbox(bbox, prev_bbox=None, frame_idx=0, roi_bounds=None):
    """
    Extract features from bounding box data
    bbox: (x1, y1, x2, y2, confidence)
    """
    x1, y1, x2, y2 = bbox[:4]
    
    # Basic bbox features
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    area = width * height
    aspect_ratio = width / (height + 1e-6)
    
    # Motion features
    if prev_bbox is not None:
        prev_x1, prev_y1, prev_x2, prev_y2 = prev_bbox[:4]
        prev_center_x = (prev_x1 + prev_x2) / 2
        prev_center_y = (prev_y1 + prev_y2) / 2
        
        dx = center_x - prev_center_x
        dy = center_y - prev_center_y
        speed = np.sqrt(dx**2 + dy**2)
        direction = np.arctan2(dy, dx)
    else:
        dx, dy, speed, direction = 0, 0, 0, 0
    
    # ROI membership (boundary detection)
    roi_flag = 0
    if roi_bounds is not None:
        roi_x1, roi_y1, roi_x2, roi_y2 = roi_bounds
        if x1 < roi_x1 or x2 > roi_x2 or y1 < roi_y1 or y2 > roi_y2:
            roi_flag = 1
    
    features = [
        x1, y1, x2, y2,  # Bbox coordinates
        center_x, center_y,  # Center
        width, height,  # Dimensions
        area, aspect_ratio,  # Shape features
        dx, dy, speed, direction,  # Motion features
        roi_flag,  # ROI flag
        frame_idx / 1000.0,  # Normalized time
    ]
    
    # Pad to match FEATURE_DIM if needed
    while len(features) < Config.FEATURE_DIM:
        features.append(0.0)
    
    return np.array(features[:Config.FEATURE_DIM], dtype=np.float32)



In [16]:

# Dataset class
class AnomalySequenceDataset(Dataset):
    def __init__(self, sequences, labels, augment=False):
        """
        sequences: list of numpy arrays of shape (T, F)
        labels: list of integer labels
        """
        self.sequences = sequences
        self.labels = labels
        self.augment = augment
        
    def __len__(self):
        return len(self.sequences)
    
    def augment_sequence(self, sequence):
        """Apply data augmentation"""
        # Temporal jitter: random frame skipping
        if np.random.random() > 0.5:
            indices = np.sort(np.random.choice(len(sequence), size=len(sequence), replace=False))
            sequence = sequence[indices]
        
        # Speed scaling
        if np.random.random() > 0.5:
            speed_factor = np.random.uniform(0.8, 1.2)
            # Scale velocity features (indices 10, 11, 12)
            sequence[:, 10:13] *= speed_factor
        
        # Horizontal flip (for left-right symmetry)
        if np.random.random() > 0.5:
            # Flip x-coordinates (indices 0, 2, 4, 10)
            max_x = 1.0  # Assuming normalized coordinates
            sequence[:, [0, 2, 4]] = max_x - sequence[:, [0, 2, 4]]
            sequence[:, 10] = -sequence[:, 10]  # dx
        
        return sequence
    
    def __getitem__(self, idx):
        sequence = self.sequences[idx].copy()
        label = self.labels[idx]
        
        if self.augment:
            sequence = self.augment_sequence(sequence)
        
        sequence = torch.from_numpy(sequence).float()
        label = torch.tensor(label, dtype=torch.long)
        
        return sequence, label



In [17]:

# RNN Model with Attention
class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1)
        
    def forward(self, lstm_output):
        # lstm_output: (batch, seq_len, hidden_dim)
        attention_weights = torch.softmax(self.attention(lstm_output), dim=1)
        # Weighted sum
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)
        return context_vector, attention_weights


In [18]:


class AnomalyDetectionRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, num_classes=6, dropout=0.3, use_attention=True):
        super(AnomalyDetectionRNN, self).__init__()
        
        self.use_attention = use_attention
        
        # Feature extractor
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Attention layer
        if use_attention:
            self.attention = AttentionLayer(hidden_dim * 2)
            classifier_input_dim = hidden_dim * 2
        else:
            classifier_input_dim = hidden_dim * 2
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(classifier_input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        # x: (batch, seq_len, features)
        batch_size, seq_len, _ = x.size()
        
        # Feature extraction
        x = self.feature_extractor(x)
        
        # LSTM
        lstm_out, _ = self.lstm(x)
        # lstm_out: (batch, seq_len, hidden_dim*2)
        
        # Attention or last output
        if self.use_attention:
            context_vector, _ = self.attention(lstm_out)
        else:
            context_vector = lstm_out[:, -1, :]
        
        # Classification
        output = self.classifier(context_vector)
        
        return output


# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training')
    for sequences, labels in pbar:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pbar.set_postfix({'loss': f'{running_loss / (pbar.n + 1):.4f}', 
                         'acc': f'{100 * correct / total:.2f}%'})
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc



In [19]:

# Validation function
def validate(model, dataloader, criterion, device, return_predictions=False):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_predictions = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for sequences, labels in pbar:
            sequences, labels = sequences.to(device), labels.to(device)
            
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            
            probs = torch.softmax(outputs, dim=1)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            if return_predictions:
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
            
            pbar.set_postfix({'loss': f'{running_loss / (pbar.n + 1):.4f}',
                            'acc': f'{100 * correct / total:.2f}%'})
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    
    if return_predictions:
        return epoch_loss, epoch_acc, all_predictions, all_labels, all_probs
    return epoch_loss, epoch_acc



In [20]:

# Evaluation metrics
def compute_metrics(y_true, y_pred, y_probs, class_names):
    """Compute comprehensive metrics"""
    
    # Classification report
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # ROC-AUC (one-vs-rest)
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    roc_auc = {}
    for i, class_name in enumerate(class_names):
        try:
            roc_auc[class_name] = roc_auc_score(y_true_bin[:, i], np.array(y_probs)[:, i])
        except:
            roc_auc[class_name] = 0.0
    
    return report, cm, roc_auc


In [21]:


# Plot confusion matrix
def plot_confusion_matrix(cm, class_names, save_path):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()



In [22]:

# Generate synthetic data for demonstration
def generate_synthetic_data(num_samples=1000, seq_length=30, feature_dim=25):
    """Generate synthetic surveillance data for testing"""
    sequences = []
    labels = []
    
    for _ in range(num_samples):
        label = np.random.randint(0, Config.NUM_CLASSES)
        
        # Generate synthetic sequence
        sequence = np.random.randn(seq_length, feature_dim).astype(np.float32)
        
        # Add label-specific patterns
        if label == 1:  # Loitering - low motion
            sequence[:, 10:13] *= 0.1
        elif label == 2:  # Boundary crossing - sudden position change
            sequence[15:, 4:6] += 0.5
        elif label == 3:  # Fall - sudden downward motion
            sequence[20:, 5] += 1.0
            sequence[20:, 11] += 1.0
        elif label == 4:  # Aggression - high speed
            sequence[:, 12] *= 2.0
        elif label == 5:  # Abandoned object - stationary
            sequence[:, 10:14] = 0
        
        sequences.append(sequence)
        labels.append(label)
    
    return sequences, labels



In [23]:
# Main training pipeline
def main():
    # ------------------------------
    # Device setup
    # ------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # ------------------------------
    # Create directories
    # ------------------------------
    os.makedirs(Config.MODEL_SAVE_PATH, exist_ok=True)
    os.makedirs(Config.RESULTS_PATH, exist_ok=True)
    
    print("="*60)
    print("RNN-Based Anomaly Detection Training Pipeline")
    print("="*60)
    
    # ------------------------------
    # Load or generate data
    # ------------------------------
    print("\n[1/6] Loading data...")
    # TODO: Replace with actual data loading from YOLOv8 + SORT outputs
    sequences, labels = generate_synthetic_data(
        num_samples=2000, 
        seq_length=Config.SEQUENCE_LENGTH,
        feature_dim=Config.FEATURE_DIM
    )
    
    print(f"Total samples: {len(sequences)}")
    print(f"Sequence shape: {sequences[0].shape}")
    print(f"Class distribution: {np.bincount(labels)}")
    
    # ------------------------------
    # Split data
    # ------------------------------
    print("\n[2/6] Splitting dataset...")
    train_seq, temp_seq, train_labels, temp_labels = train_test_split(
        sequences, labels, test_size=(Config.VAL_SPLIT + Config.TEST_SPLIT), 
        random_state=42, stratify=labels
    )
    
    val_seq, test_seq, val_labels, test_labels = train_test_split(
        temp_seq, temp_labels, 
        test_size=Config.TEST_SPLIT/(Config.VAL_SPLIT + Config.TEST_SPLIT),
        random_state=42, stratify=temp_labels
    )
    
    print(f"Train: {len(train_seq)}, Val: {len(val_seq)}, Test: {len(test_seq)}")
    
    # ------------------------------
    # Create datasets & dataloaders
    # ------------------------------
    train_dataset = AnomalySequenceDataset(train_seq, train_labels, augment=True)
    val_dataset = AnomalySequenceDataset(val_seq, val_labels, augment=False)
    test_dataset = AnomalySequenceDataset(test_seq, test_labels, augment=False)
    
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=2)
    
    # ------------------------------
    # Initialize model
    # ------------------------------
    print("\n[3/6] Initializing model...")
    model = AnomalyDetectionRNN(
        input_dim=Config.FEATURE_DIM,
        hidden_dim=Config.HIDDEN_DIM,
        num_layers=Config.NUM_LAYERS,
        num_classes=Config.NUM_CLASSES,
        dropout=Config.DROPOUT,
        use_attention=True
    ).to(device)
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # ------------------------------
    # Loss, optimizer, scheduler
    # ------------------------------
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=Config.LR_REDUCE_PATIENCE, factor=0.5
    )
    
    # ------------------------------
    # Training loop
    # ------------------------------
    print("\n[4/6] Training model...")
    best_val_acc = 0.0
    patience_counter = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    # Helper: serialize config safely
    def serialize_config():
        return {
            'SEQUENCE_LENGTH': Config.SEQUENCE_LENGTH,
            'FEATURE_DIM': Config.FEATURE_DIM,
            'HIDDEN_DIM': Config.HIDDEN_DIM,
            'NUM_LAYERS': Config.NUM_LAYERS,
            'NUM_CLASSES': Config.NUM_CLASSES,
            'DROPOUT': Config.DROPOUT,
            'BATCH_SIZE': Config.BATCH_SIZE,
            'LEARNING_RATE': Config.LEARNING_RATE,
            'NUM_EPOCHS': Config.NUM_EPOCHS,
            'EARLY_STOP_PATIENCE': Config.EARLY_STOP_PATIENCE,
            'LR_REDUCE_PATIENCE': Config.LR_REDUCE_PATIENCE,
            'TRAIN_SPLIT': Config.TRAIN_SPLIT,
            'VAL_SPLIT': Config.VAL_SPLIT,
            'TEST_SPLIT': Config.TEST_SPLIT,
            'CLASS_NAMES': Config.CLASS_NAMES,
            'Dataset': Config.DATASET_PATH,
            'MODEL_SAVE_PATH': Config.MODEL_SAVE_PATH,
            'RESULTS_PATH': Config.RESULTS_PATH
        }
    
    for epoch in range(Config.NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{Config.NUM_EPOCHS}")
        print("-" * 60)
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        scheduler.step(val_loss)
        
        # Early stopping and model saving
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'val_loss': val_loss,
                'config': serialize_config()
            }, os.path.join(Config.MODEL_SAVE_PATH, 'rnn_anomaly_best.pt'))
            print(f"✓ Best model saved (Val Acc: {val_acc:.2f}%)")
        else:
            patience_counter += 1
            if patience_counter >= Config.EARLY_STOP_PATIENCE:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    # ------------------------------
    # Evaluate on test set
    # ------------------------------
    print("\n[5/6] Evaluating on test set...")
    checkpoint = torch.load(os.path.join(Config.MODEL_SAVE_PATH, 'rnn_anomaly_best.pt'), map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    test_loss, test_acc, y_pred, y_true, y_probs = validate(
        model, test_loader, criterion, device, return_predictions=True
    )
    
    print(f"\nTest Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    
    # Compute metrics
    report, cm, roc_auc = compute_metrics(y_true, y_pred, y_probs, Config.CLASS_NAMES)
    
    print("\nClassification Report:")
    print(json.dumps(report, indent=2))
    
    print("\nROC-AUC Scores:")
    for class_name, score in roc_auc.items():
        print(f"  {class_name}: {score:.4f}")
    
    # Plot confusion matrix
    plot_confusion_matrix(
        cm, Config.CLASS_NAMES, os.path.join(Config.RESULTS_PATH, 'confusion_matrix.png')
    )
    
    # ------------------------------
    # Save results
    # ------------------------------
    print("\n[6/6] Saving results...")
    results = {
        'test_accuracy': test_acc,
        'test_loss': test_loss,
        'classification_report': report,
        'roc_auc': roc_auc,
        'confusion_matrix': cm.tolist(),
        'training_history': history,
        'best_val_acc': best_val_acc
    }
    
    with open(os.path.join(Config.RESULTS_PATH, 'metrics.json'), 'w') as f:
        json.dump(results, f, indent=4)
    
    # Save final model
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': serialize_config(),
        'class_names': Config.CLASS_NAMES
    }, os.path.join(Config.MODEL_SAVE_PATH, 'rnn_anomaly_final.pt'))
    
    print("\n" + "="*60)
    print("Training completed successfully!")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"Test accuracy: {test_acc:.2f}%")
    print(f"\nModels saved in: {Config.MODEL_SAVE_PATH}")
    print(f"Results saved in: {Config.RESULTS_PATH}")
    print("="*60)


if __name__ == "__main__":
    main()


Using device: cpu
RNN-Based Anomaly Detection Training Pipeline

[1/6] Loading data...
Total samples: 2000
Sequence shape: (30, 25)
Class distribution: [323 345 320 348 338 326]

[2/6] Splitting dataset...
Train: 1400, Val: 300, Test: 300

[3/6] Initializing model...
Model parameters: 679,879

[4/6] Training model...

Epoch 1/100
------------------------------------------------------------


Validation: 100%|██████████| 10/10 [00:00<00:00, 21.76it/s, loss=1.9914, acc=16.00%]


Train Loss: 1.7952, Train Acc: 16.00%
Val Loss: 1.7923, Val Acc: 16.00%
Learning Rate: 0.000100
✓ Best model saved (Val Acc: 16.00%)

Epoch 2/100
------------------------------------------------------------


Validation: 100%|██████████| 10/10 [00:00<00:00, 25.50it/s, loss=1.7891, acc=17.00%]


Train Loss: 1.7925, Train Acc: 18.43%
Val Loss: 1.7891, Val Acc: 17.00%
Learning Rate: 0.000100
✓ Best model saved (Val Acc: 17.00%)

Epoch 3/100
------------------------------------------------------------


...

Train Loss: 0.2264, Train Acc: 91.64%
Val Loss: 0.3072, Val Acc: 89.33%
Learning Rate: 0.000025

Epoch 88/100
------------------------------------------------------------


Validation: 100%|██████████| 10/10 [00:00<00:00, 20.54it/s, loss=0.4498, acc=89.00%]


Train Loss: 0.2105, Train Acc: 92.50%
Val Loss: 0.3598, Val Acc: 89.00%
Learning Rate: 0.000025

Early stopping triggered after 88 epochs

[5/6] Evaluating on test set...


Validation: 100%|██████████| 10/10 [00:00<00:00, 25.18it/s, loss=0.3037, acc=90.00%]



Test Loss: 0.2733, Test Acc: 90.00%

Classification Report:
{
  "Normal": {
    "precision": 0.8333333333333334,
    "recall": 0.7142857142857143,
    "f1-score": 0.7692307692307692,
    "support": 49
  },
  "Loitering": {
    "precision": 0.9803921568627451,
    "recall": 0.9615384615384616,
    "f1-score": 0.970873786407767,
    "support": 52
  },
  "Boundary_Crossing": {
    "precision": 0.72,
    "recall": 0.75,
    "f1-score": 0.7346938775510204,
    "support": 48
  },
  "Fall": {
    "precision": 0.9107142857142857,
    "recall": 0.9807692307692307,
    "f1-score": 0.9444444444444444,
    "support": 52
  },
  "Aggression": {
    "precision": 0.9803921568627451,
    "recall": 1.0,
    "f1-score": 0.99009900990099,
    "support": 50
  },
  "Abandoned_Object": {
    "precision": 0.96,
    "recall": 0.9795918367346939,
    "f1-score": 0.9696969696969697,
    "support": 49
  },
  "accuracy": 0.9,
  "macro avg": {
    "precision": 0.8974719887955181,
    "recall": 0.8976975405546835,
